In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
%run ./Column_autosize.ipynb
import win32com.client as win32
from jinja2 import Template
import sqlite3
import datetime

In [56]:
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Effectif_Data_Quality.xlsx", converters={'Matricule':str}) # effectif présent
titre_sej = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Titreséjour.xlsx")
nationality = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Table_Nationalité.xlsx")

### Identification des salariés etrangers

In [57]:
UE = ["DE","BE","BG","HR","DK","ES","FR","GR","NL","IE","IT","LU","MC","NC","PT","CZ","RO","GB","RS","SE","CH","CY","FI","HU","LV","LT","SI"]
nationality_active = nationality.loc[nationality["Date de fin"].isna()==True]
nationality_fr = nationality_active.loc[nationality_active["Pays"]=="FR", "Ressource"]
nationality_fr = nationality_fr.unique()
strangers = nationality_active.loc[(nationality_active["Ressource"].isin(nationality_fr)==False) & (nationality_active["Pays"].isin(UE)==False), "Ressource"].unique()

### Recherche des etrangers sans titres de séjour

In [58]:
salarie_titre = titre_sej["Ressource"].unique()
sans_titre = [sal for sal in strangers if sal not in salarie_titre] # Salariés etrangers n'ayant pas de titre dans PN
salarie_notitre = fafr.loc[fafr["Matricule"].astype("int64").isin(sans_titre)==True]
salarie_notitre["Anomalie"] = "Salarié sans titre de séjour"
salarie_notitre.shape

<ipython-input-58-0e8b6ee5a2d6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salarie_notitre["Anomalie"] = "Salarié sans titre de séjour"


(85, 64)

### Recherche des salariés etrangers ayant un titre expiré

In [59]:
etrangers_titre = [ sal for sal in strangers if sal not in sans_titre] # Exclusion parmis les etrangers des salariés sans titres
etrangers_titre = [sal for sal in etrangers_titre if sal in list(fafr["Matricule"].astype("int64")) ] # On conserve que les salariés présents
titre_etr = titre_sej.loc[titre_sej["Ressource"].isin(etrangers_titre)==True].copy()
titre_etr["today"] = datetime.datetime.today().date()
titre_etr.columns = ['Matricule', 'Date_de_début', 'Date_de_fin', 'Nº_de_carte_étranger','today']
titre_etr.loc[titre_etr["Date_de_fin"].isna()==True, "Date_de_fin"] = datetime.datetime(2050, 1, 1)

In [60]:
# Création d'une base de données in memory
conn = sqlite3.connect(':memory:')

# Ajout de info_comp comme table de la base de données
titre_etr.to_sql('titre_etr', conn, index=False)
req = "Select Matricule, Date_de_début, Date_de_fin, today from titre_etr where today between Date_de_début and Date_de_fin "

# Exécution de la requête et stokage du résultat dans un DataFrame
df = pd.read_sql_query(req, conn)
salarie_off_titre = [sal for sal in titre_etr["Matricule"] if sal not in list(df["Matricule"])]
salarie_exp_titre = fafr.loc[fafr["Matricule"].astype("int64").isin(salarie_off_titre)].copy()
salarie_exp_titre["Anomalie"] = "Titre de séjour expiré"
salarie_notitre = salarie_notitre.append(salarie_exp_titre)
salarie_notitre.shape

(138, 64)

### Recherche des salariés sans date de fin

In [61]:
sal_nodate = titre_etr.loc[titre_etr["Date_de_fin"]==datetime.datetime(2050, 1, 1)]
res_nodate = fafr.loc[fafr["Matricule"].astype("int64").isin(list(sal_nodate["Matricule"]))].copy()
res_nodate["Anomalie"] = "Date de fin du titre de séjour absente"
salarie_notitre = salarie_notitre.append(res_nodate)

In [62]:
salarie_notitre.columns = [col.replace(" ", "_") for col in salarie_notitre.columns]

In [64]:
salarie_notitre = salarie_notitre[["Matricule", "Nom", "Prénom", "Nom_RRH_Direct", "Prénom_RRH_Direct"]]

In [65]:
salarie_notitre

,Matricule,Nom,Prénom,Nom_RRH_Direct,Prénom_RRH_Direct
328,12287,YILMAZ,ETEM,PERRIN,OLIVIA
905,30563,YELLES CHAOUCHE,MANIL,LOMBARDI,BRUNO
991,31564,SYLLA,ADAMA IBRAHIMA,JEMRICH,MARYSE
1003,32022,MAKAMONA MBUMBA,CESAR,DENIS,JEAN-CHRISTOPHE
1041,32704,JUNYSBEKOVA,AIGERIM,BESSAA,JASMINE
...,...,...,...,...,...
2608,419404,KHECHAB,OUMAIMA,SATTA,LAURA
4404,415651,OMPOUENGUI OGANDAGA,CHRIS ALPHEN,PALLEY,NICOLAS
6714,418072,JARI,MERIEM,JEMRICH,MARYSE
7379,414833,MEGZARI,ABDELHAK,BRAHMI,SAFIA
